In [1]:
%load_ext autoreload
%autoreload 2
%pwd

'/Users/luis/uni/dl-vc/voxl3r'

In [2]:
# setup PYTHONPATH
import sys
sys.path += ['.', './extern/scannetpp', './extern/mast3r', './extern/mast3r/dust3r']

In [3]:
from pathlib import Path
import torch
from torchvision.io import read_image

import pyvista as pv
pv.set_jupyter_backend('client')

from dataset import SceneDataset, SceneDatasetTransformToTorch
from einops import rearrange
from models.surface_net_baseline.model import SimpleOccNetConfig
from models.surface_net_baseline.module import LRConfig, OccSurfaceNet, OptimizerConfig
from utils.data_parsing import load_yaml_munch
from lightning import Trainer
from lightning.pytorch.loggers import WandbLogger
import lightning as pl

from utils.visualize import visualize_mesh
from utils.basic import get_default_device


In [4]:
import numpy as np

np.random.seed(42)
# Init dataset and load for visualization
dataset_tdf = SceneDataset(
    data_dir="datasets/scannetpp/data",
    camera="iphone",
    n_points=300000,
    threshold_occ=0.01,
    representation="tdf",
    visualize=True,
    max_seq_len=10,
)

dataset = SceneDataset(
    data_dir="datasets/scannetpp/data",
    camera="iphone",
    n_points=300000,
    threshold_occ=0.01,
    representation="occ",
    visualize=True,
    max_seq_len=10,
)

device = get_default_device()
transform = SceneDatasetTransformToTorch(target_device=device)

In [5]:
idx = dataset_tdf.get_index_from_scene("8b2c0938d6")
data = dataset_tdf[idx]
data_occ = dataset[idx]
data = dataset_tdf[idx]
data_occ = dataset[idx]

## Scene Chunking

In [6]:
image_names, camera_params_list, camera_center = data["images"]

visualize_mesh(
        data["mesh"],
        images=image_names,
        camera_params_list=camera_params_list,
        point_coords=camera_center,
    )

Widget(value='<iframe src="http://localhost:65377/index.html?ui=P_0x2aa8e42f0_0&reconnect=auto" class="pyvista…

## Distance Function Computation

In [7]:
from dataset import plot_training_example

plot_training_example(data, idx)

Widget(value='<iframe src="http://localhost:65377/index.html?ui=P_0x2ab582360_1&reconnect=auto" class="pyvista…

## Visualization: Transform to Voxel Grid

In [8]:
from dataset import plot_occupency_grid

plot_occupency_grid(data_occ, idx, resolution=0.015)

Voxel Grid Size (L, W, H): (np.float64(2.566579868864046), np.float64(2.2563111128633584), np.float64(1.3567202929135482))
Voxel Grid Center: [3.4658195  2.14701433 1.45982352]
Occupied Voxels: 24394.0


Widget(value='<iframe src="http://localhost:65377/index.html?ui=P_0x2ab64f230_2&reconnect=auto" class="pyvista…

## Project RGB Values to Scene

In [9]:
from models.surface_net_baseline.train import visualize_unprojection

visualize_unprojection(data_occ)

/Users/luis/uni/dl-vc/voxl3r/dataset.py:293: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  points = torch.tensor(torch.from_numpy(points).float()).to(self.target_device)
/Users/luis/uni/dl-vc/voxl3r/dataset.py:294: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  gt = torch.tensor(torch.from_numpy(gt).float()).to(self.target_device)
/Users/luis/uni/dl-vc/voxl3r/.venv/lib/python3.12/site-packages/torch/nn/functional.py:4902: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/Users/luis/uni/dl-vc/voxl3r/mod

Widget(value='<iframe src="http://localhost:65377/index.html?ui=P_0x2b149b620_3&reconnect=auto" class="pyvista…

## First Training Run Naive Approach

In [ ]:
from models.surface_net_baseline.data import OccSurfaceNetDatamodule
from utils.visualize import plot_voxel_grid

max_seq_len = 10
scene_dataset = SceneDataset(data_dir="datasets/scannetpp/data", camera="iphone", n_points=300000, threshold_occ=0.01, representation="occ", visualize=True, max_seq_len=max_seq_len)
scene_dataset.seed = 42
model_path = ".lightning/occ-surface-net/surface-net-baseline/r3cf6z5t/checkpoints/epoch=262-step=24459.00-val_accuracy=0.89.ckpt"

model = OccSurfaceNet.load_from_checkpoint(model_path)
datamodule = OccSurfaceNetDatamodule(scene_dataset, "8b2c0938d6", batch_size=128, max_sequence_length=max_seq_len)
datamodule.setup('test')
test_dict = model.test_visualize(datamodule.test_dataloader())

gt = torch.cat(test_dict['gt'])
points = torch.cat(test_dict['points'])
y = torch.sigmoid(torch.cat(test_dict['out']))
#y[y < 0.5] = 0.0
#y[y > 0.5] = 1.0

mesh = Path(scene_dataset.data_dir) / scene_dataset.scenes[datamodule.scene_idx] / "scans" / "mesh_aligned_0.05.ply"
visualize_mesh(mesh, point_coords=points.detach().cpu().numpy(), heat_values=y.detach().cpu().numpy())
#plot_voxel_grid(points.detach().cpu().numpy(), y.detach().cpu().numpy(), ref_mesh=mesh)

/Users/luis/uni/dl-vc/voxl3r/dataset.py:293: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  points = torch.tensor(torch.from_numpy(points).float()).to(self.target_device)
/Users/luis/uni/dl-vc/voxl3r/dataset.py:294: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  gt = torch.tensor(torch.from_numpy(gt).float()).to(self.target_device)
/Users/luis/uni/dl-vc/voxl3r/.venv/lib/python3.12/site-packages/torch/nn/functional.py:4902: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


Widget(value='<iframe src="http://localhost:65377/index.html?ui=P_0x2b1472090_4&reconnect=auto" class="pyvista…

In [11]:
occ = y.clone()
occ[y < 0.5] = 0.0
occ[y > 0.5] = 1.0
plot_voxel_grid(points.detach().cpu().numpy(), occ.detach().cpu().numpy(), ref_mesh=mesh)

Voxel Grid Size (L, W, H): (np.float32(2.36118), np.float32(2.178124), np.float32(1.2722685))
Voxel Grid Center: [3.4625769 2.1568832 1.4368578]
Occupied Voxels: 13377.0


Widget(value='<iframe src="http://localhost:65377/index.html?ui=P_0x2b1c78c50_5&reconnect=auto" class="pyvista…